[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abossenbroek/3d-from-2d-reconstruction/blob/main/labs/mesh_rcnn/mesh_r_cnn_demo.ipynb)

# Example of using Mesh R-CNN
We give a simple example of how to run [Mesh R-CNN](https://openaccess.thecvf.com/content_ICCV_2019/papers/Gkioxari_Mesh_R-CNN_ICCV_2019_paper.pdf) in a notebook. We will upload an image and show the fitted mesh.

In [ ]:
import os
import sys
import torch

need_pytorch3d = False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d = True
if need_pytorch3d:
    if torch.__version__.startswith("1.13.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str = torch.__version__.split("+")[0].replace(".", "")
        version_str = "".join(
            [
                f"py3{sys.version_info.minor}_cu",
                torch.version.cuda.replace(".", ""),
                f"_pyt{pyt_version_str}",
            ]
        )
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

## 0. Install and load dependencies

In [ ]:
import glob
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.io import load_objs_as_meshes

In [ ]:
%%shell
python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
git clone https://github.com/facebookresearch/meshrcnn.git
cd meshrcnn && pip install -e .

## 2. Upload an image

In [ ]:
uploaded = files.upload()

In [ ]:
import os

image_base_name = list(uploaded.keys())[0].split(".jpg")[0]

os.environ["INPUT_IMAGE_BASE"] = image_base_name
os.environ["INPUT_IMAGE"] = os.path.join("..", list(uploaded.keys())[0])

## 3. Run prediction

In [ ]:
%%shell

cd meshrcnn
python demo/demo.py \
--config-file configs/pix3d/meshrcnn_R50_FPN.yaml \
--input $INPUT_IMAGE \
--output output_demo \
--onlyhighest MODEL.WEIGHTS meshrcnn://meshrcnn_R50.pth


## 4. Show predictions
First we will show the files that were generated

In [ ]:
!ls meshrcnn/output_demo/$INPUT_IMAGE_BASE

In [ ]:
img_path = os.path.join("meshrcnn/output_demo", image_base_name)

for img in glob.glob(img_path + "/" + "*png"):
    img = cv2.imread(img)  # reads image
    cv2_imshow(img)

In [ ]:
plots = {}

for mesh_file in glob.glob(img_path + "/" + "*obj"):
    mesh = load_objs_as_meshes([mesh_file], load_textures=False)  # reads mesh
    plots[mesh_file] = {"mesh": mesh[0]}

In [ ]:
plot_scene(plots)